In [1]:
import AdventOfCode
import qualified Data.Vector as V

In [2]:
input <- dayString 7

instructions = read ("[" ++ input ++ "]") :: [Int]

In [3]:
data Opcode = Opcode
    { opcodeCode  :: Int
    , opcodeMode1 :: Bool
    , opcodeMode2 :: Bool
    , opcodeMode3 :: Bool
    } deriving (Eq, Show)

In [4]:
parseCode :: Int -> Opcode
parseCode i = let
    m3   = i `div` 10000
    m2   = (i - m3*10000) `div` 1000
    m1   = (i - m3*10000 - m2*1000) `div` 100
    code = i `rem` 100
    in Opcode code (m1==1) (m2==1) (m3==1)

In [5]:
step :: V.Vector Int -> [Int] -> [Int] -> Int -> (V.Vector Int, [Int], [Int], Int)
step program input output index = do
    let opcode = parseCode (program V.! index)
    case opcodeCode opcode of
        1 -> let
            a = if opcodeMode1 opcode then program V.! (index+1) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then program V.! (index+2) else program V.! (program V.! (index+2))
            o = program V.! (index+3)
            p = program V.// [(o,a+b)]
            in (p, input, output, index+4)
        2 -> let
            a = if opcodeMode1 opcode then program V.! (index+1) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then program V.! (index+2) else program V.! (program V.! (index+2))
            o = program V.! (index+3)
            p = program V.// [(o,a*b)]
            in (p, input, output, index+4)
        3 -> let
            v = head input
            input' = tail input
            o = program V.! (index+1)
            p = program V.// [(o, v)]
            in (p, input', output, index+2)
        4 -> let
            a = if opcodeMode1 opcode then program V.! (index+1) else program V.! (program V.! (index+1))
            output' = a:output
            in (program, input, output', index+2)
        5 -> let
            a = if opcodeMode1 opcode then program V.! (index+1) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then program V.! (index+2) else program V.! (program V.! (index+2))
            in if a/=0 then (program, input, output, b) else (program, input, output, index+3)
        6 -> let
            a = if opcodeMode1 opcode then program V.! (index+1) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then program V.! (index+2) else program V.! (program V.! (index+2))
            in if a==0 then (program, input, output, b) else (program, input, output, index+3)
        7 -> let
            a = if opcodeMode1 opcode then program V.! (index+1) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then program V.! (index+2) else program V.! (program V.! (index+2))
            o = program V.! (index+3)
            in if a < b then (program V.// [(o,1)], input, output, index+4) else (program V.// [(o, 0)], input, output, index+4)
        8 -> let
            a = if opcodeMode1 opcode then program V.! (index+1) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then program V.! (index+2) else program V.! (program V.! (index+2))
            o = program V.! (index+3)
            in if a == b then (program V.// [(o,1)], input, output, index+4) else (program V.// [(o, 0)], input, output, index+4)
        99 -> (program, input, output, index)

Line 6: Reduce duplication
Found:
a = if opcodeMode1 opcode then program V.! (index + 1) else
      program V.! (program V.! (index + 1))
b = if opcodeMode2 opcode then program V.! (index + 2) else
      program V.! (program V.! (index + 2))
o = program V.! (index + 3)

Why not:
Combine with -:12:13Line 6: Reduce duplication
Found:
a = if opcodeMode1 opcode then program V.! (index + 1) else
      program V.! (program V.! (index + 1))
b = if opcodeMode2 opcode then program V.! (index + 2) else
      program V.! (program V.! (index + 2))
o = program V.! (index + 3)

Why not:
Combine with -:36:13Line 6: Reduce duplication
Found:
a = if opcodeMode1 opcode then program V.! (index + 1) else
      program V.! (program V.! (index + 1))
b = if opcodeMode2 opcode then program V.! (index + 2) else
      program V.! (program V.! (index + 2))
o = program V.! (index + 3)

Why not:
Combine with -:41:13

In [6]:
loop program input output index = let
    (program', input', output', index') = step program input output index
    in if index == index' then (program', output', index) else loop program' input' output' index'

In [7]:
run input = let (_, o, _) = loop (V.fromList instructions) input [] 0 in o

In [8]:
import Data.List
import Data.Function.Memoize
import Data.Function

In [9]:
run' = memoize run

In [10]:
chain = foldl' (\value n -> run' (n:value))

In [11]:
maximumBy (compare `on` fst) $ map (\p -> (head $ chain [0] p, p)) $ permutations [0..4]

(75228,[0,3,4,2,1])

In [12]:
-- import Debug.Trace

traceShow = flip const

loop' program input output index = let
    (program', input', output', index') = step program input output index
    in if output' /= output
        then Right (program', output', index')
        else if index == index'
            then Left (head output')
            else loop' program' input' output' index'

starting [s1, s2, s3, s4, s5] [(p1, i1), (p2, i2), (p3, i3), (p4, i4), (p5, i5)] start =
    case loop' p1 [s1, start] [] i1 of
        Left{} -> start
        r1@(Right (p1', [o1], i1')) -> traceShow r1 $ case loop' p2 [s2, o1] [] i2 of
            Left{} -> start
            r2@(Right (p2', [o2], i2')) -> traceShow r2 $ case loop' p3 [s3, o2] [] i3 of
                Left{} -> start
                r3@(Right (p3', [o3], i3')) -> traceShow r3 $ case loop' p4 [s4, o3] [] i4 of
                    Left{} -> start
                    r4@(Right (p4', [o4], i4')) -> traceShow r4 $ case loop' p5 [s5, o4] [] i5 of
                        Left{} -> start
                        r5@(Right (p5', [o5], i5')) -> traceShow r5 $ feedback [(p1',i1'), (p2',i2'), (p3',i3'), (p4',i4'), (p5',i5')] o5

feedback [(p1, i1), (p2, i2), (p3, i3), (p4, i4), (p5, i5)] start =
    case loop' p1 [start] [] i1 of
        Left{} -> start
        r1@(Right (p1', [o1], i1')) -> traceShow r1 $ case loop' p2 [o1] [] i2 of
            Left{} -> start
            r2@(Right (p2', [o2], i2')) -> traceShow r2 $ case loop' p3 [o2] [] i3 of
                Left{} -> start
                r3@(Right (p3', [o3], i3')) -> traceShow r3 $ case loop' p4 [o3] [] i4 of
                    Left{} -> start
                    r4@(Right (p4', [o4], i4')) -> traceShow r4 $ case loop' p5 [o4] [] i5 of
                        Left{} -> start
                        r5@(Right (p5', [o5], i5')) -> traceShow r5 $ feedback [(p1',i1'), (p2',i2'), (p3',i3'), (p4',i4'), (p5',i5')] o5

In [13]:
initialise p ss = starting ss (replicate 5 (p,0)) 0

In [14]:
p =(V.fromList [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5])

Line 1: Redundant bracket
Found:
(V.fromList
   [3, 26, 1001, 26, -4, 26, 3, 27, 1002, 27, 2, 27, 1, 27, 26, 27, 4,
    27, 1001, 28, -1, 28, 1005, 28, 6, 99, 0, 0, 5])
Why not:
V.fromList
  [3, 26, 1001, 26, -4, 26, 3, 27, 1002, 27, 2, 27, 1, 27, 26, 27, 4,
   27, 1001, 28, -1, 28, 1005, 28, 6, 99, 0, 0, 5]

In [17]:
maximum $ map (initialise (V.fromList instructions)) $ permutations [5..9]

79846026